In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv(r"H:\Nasa Space Apps 2025\data\cumulative_2025.09.28_05.59.15.csv")
data.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,koi_fpflag_nt,...,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,...,0.200,0.160,0.200,0.170,0.080,0.130,0.310,0.170,0.320,0.160
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,Done,2018-08-16,CANDIDATE,0.969,0,...,0.000,0.480,0.390,0.360,0.490,0.340,0.120,0.730,0.500,0.450
2,3,10811496,K00753.01,NaN,CANDIDATE,Done,2018-08-16,CANDIDATE,0.000,0,...,-0.034,0.070,0.042,0.072,0.002,0.071,-0.027,0.074,0.027,0.074
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,Done,2018-08-16,FALSE POSITIVE,0.000,0,...,0.147,0.078,0.289,0.079,-0.257,0.072,0.099,0.077,0.276,0.076
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,...,-0.090,0.180,0.100,0.140,0.070,0.180,0.020,0.160,0.070,0.200


In [3]:
data.columns

Index(['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition',
       'koi_vet_stat', 'koi_vet_date', 'koi_pdisposition', 'koi_score',
       'koi_fpflag_nt',
       ...
       'koi_dicco_mdec', 'koi_dicco_mdec_err', 'koi_dicco_msky',
       'koi_dicco_msky_err', 'koi_dikco_mra', 'koi_dikco_mra_err',
       'koi_dikco_mdec', 'koi_dikco_mdec_err', 'koi_dikco_msky',
       'koi_dikco_msky_err'],
      dtype='object', length=141)

In [4]:
koi_columns = [
    # Identity & disposition
    "kepid",
    "kepoi_name",
    "koi_disposition",
    "koi_pdisposition",
    "koi_score",

    # Core transit parameters
    "koi_period",
    "koi_duration",
    "koi_depth",
    "koi_time0bk",
    "koi_prad",
    "koi_ror",       # Rp/Rs
    "koi_dor",       # a/Rs
    "koi_impact",
    "koi_incl",

    # Stellar parameters
    "koi_srad",
    "koi_smass",
    "koi_steff",
    "koi_slogg",
    "koi_smet",
    "koi_kepmag",

    # False-positive diagnostic flags
    "koi_fpflag_nt", # Not transit-like
    "koi_fpflag_ss", # Stellar eclipse
    "koi_fpflag_co", # Centroid offset
    "koi_fpflag_ec", # Ephemeris match contamination

    # Vetting & provenance
    "koi_vet_stat",
    "koi_vet_date",
    "koi_disp_prov",

    "koi_period_err1",
    "koi_period_err2",
    "koi_duration_err1",
    "koi_duration_err2",
    "koi_depth_err1",
    "koi_depth_err2",
    "koi_prad_err1",
    "koi_prad_err2",
    "koi_steff_err1",
    "koi_steff_err2"
]


In [5]:
data = data[koi_columns]

In [6]:
data.columns

Index(['kepid', 'kepoi_name', 'koi_disposition', 'koi_pdisposition',
       'koi_score', 'koi_period', 'koi_duration', 'koi_depth', 'koi_time0bk',
       'koi_prad', 'koi_ror', 'koi_dor', 'koi_impact', 'koi_incl', 'koi_srad',
       'koi_smass', 'koi_steff', 'koi_slogg', 'koi_smet', 'koi_kepmag',
       'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_vet_stat', 'koi_vet_date', 'koi_disp_prov', 'koi_period_err1',
       'koi_period_err2', 'koi_duration_err1', 'koi_duration_err2',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad_err1', 'koi_prad_err2',
       'koi_steff_err1', 'koi_steff_err2'],
      dtype='object')

In [7]:
data.head()

,kepid,kepoi_name,koi_disposition,koi_pdisposition,koi_score,koi_period,koi_duration,koi_depth,koi_time0bk,koi_prad,...,koi_period_err1,koi_period_err2,koi_duration_err1,koi_duration_err2,koi_depth_err1,koi_depth_err2,koi_prad_err1,koi_prad_err2,koi_steff_err1,koi_steff_err2
0,10797460,K00752.01,CONFIRMED,CANDIDATE,1.000,9.488036,2.95750,615.8,170.538750,2.26,...,2.775000e-05,-2.775000e-05,0.08190,-0.08190,19.5,-19.5,0.26,-0.15,81.0,-81.0
1,10797460,K00752.02,CONFIRMED,CANDIDATE,0.969,54.418383,4.50700,874.8,162.513840,2.83,...,2.479000e-04,-2.479000e-04,0.11600,-0.11600,35.5,-35.5,0.32,-0.19,81.0,-81.0
2,10811496,K00753.01,CANDIDATE,CANDIDATE,0.000,19.899140,1.78220,10829.0,175.850252,14.60,...,1.494000e-05,-1.494000e-05,0.03410,-0.03410,171.0,-171.0,3.92,-1.31,158.0,-176.0
3,10848459,K00754.01,FALSE POSITIVE,FALSE POSITIVE,0.000,1.736952,2.40641,8079.2,170.307565,33.46,...,2.630000e-07,-2.630000e-07,0.00537,-0.00537,12.8,-12.8,8.50,-2.83,157.0,-174.0
4,10854555,K00755.01,CONFIRMED,CANDIDATE,1.000,2.525592,1.65450,603.3,171.595550,2.75,...,3.761000e-06,-3.761000e-06,0.04200,-0.04200,16.9,-16.9,0.88,-0.35,169.0,-211.0


In [8]:
data.isna().sum()

kepid                   0
kepoi_name              0
koi_disposition         0
koi_pdisposition        0
koi_score            1510
koi_period              0
koi_duration            0
koi_depth             363
koi_time0bk             0
koi_prad              363
koi_ror               363
koi_dor               363
koi_impact            363
koi_incl              364
koi_srad              363
koi_smass             363
koi_steff             363
koi_slogg             363
koi_smet              386
koi_kepmag              1
koi_fpflag_nt           0
koi_fpflag_ss           0
koi_fpflag_co           0
koi_fpflag_ec           0
koi_vet_stat            0
koi_vet_date            0
koi_disp_prov           0
koi_period_err1       454
koi_period_err2       454
koi_duration_err1     454
koi_duration_err2     454
koi_depth_err1        454
koi_depth_err2        454
koi_prad_err1         363
koi_prad_err2         363
koi_steff_err1        468
koi_steff_err2        483
dtype: int64

In [9]:
data['koi_disposition'].value_counts()

koi_disposition
FALSE POSITIVE    4839
CONFIRMED         2746
CANDIDATE         1979
Name: count, dtype: int64

In [10]:
data['koi_pdisposition'].value_counts()

koi_pdisposition
FALSE POSITIVE    4847
CANDIDATE         4717
Name: count, dtype: int64

In [11]:
numeric_columns = data.select_dtypes(include=['number']).columns.tolist()
len(numeric_columns)

31

In [12]:
columns_to_impute = [
    "koi_score", "koi_depth", "koi_prad", "koi_ror", "koi_dor", "koi_impact", "koi_incl",
    "koi_srad", "koi_smass", "koi_steff", "koi_slogg", "koi_smet",
    "koi_period_err1", "koi_period_err2", "koi_duration_err1", "koi_duration_err2",
    "koi_depth_err1", "koi_depth_err2", "koi_prad_err1", "koi_prad_err2",
    "koi_steff_err1", "koi_steff_err2"
]

In [13]:
rf_imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, random_state=42),
                              max_iter=10, random_state=42)

data[columns_to_impute] = rf_imputer.fit_transform(data[columns_to_impute])

C:\Users\CRIZMA\AppData\Roaming\Python\Python312\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [14]:
data.isna().sum()

kepid                0
kepoi_name           0
koi_disposition      0
koi_pdisposition     0
koi_score            0
koi_period           0
koi_duration         0
koi_depth            0
koi_time0bk          0
koi_prad             0
koi_ror              0
koi_dor              0
koi_impact           0
koi_incl             0
koi_srad             0
koi_smass            0
koi_steff            0
koi_slogg            0
koi_smet             0
koi_kepmag           1
koi_fpflag_nt        0
koi_fpflag_ss        0
koi_fpflag_co        0
koi_fpflag_ec        0
koi_vet_stat         0
koi_vet_date         0
koi_disp_prov        0
koi_period_err1      0
koi_period_err2      0
koi_duration_err1    0
koi_duration_err2    0
koi_depth_err1       0
koi_depth_err2       0
koi_prad_err1        0
koi_prad_err2        0
koi_steff_err1       0
koi_steff_err2       0
dtype: int64

In [15]:
data.shape

(9564, 37)

In [16]:
data.to_csv('cleaned_data.csv', index=False)